# Práctico 1 - Parte 2

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

[Documentación de Pytorch](https://pytorch.org/docs/stable/index.html)

[Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

En esta tercera parte se arman los datasets, los dataloaders y se entrena y prueba el modelo.

## Importaciones

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from gensim import corpora
from tqdm.notebook import tqdm
from sklearn.metrics import balanced_accuracy_score
from practico1_modulo import *

## Constantes

In [2]:
EPOCHS = 1
BATCH_SIZE = 100

## Carga de datos

Carga de datos de entrenamiento

In [3]:
X_train = torch.load('./data/X_train.pt')
y_train = torch.load('./data/y_train.pt')

In [4]:
# La reducción del dataset de entrenamiento es temporal
# Cuando compruebe que funciona se eliminará esta celda.
X_train = X_train[:1000000]
X_train.shape

torch.Size([1000000, 17])

In [5]:
# La reducción del dataset de entrenamiento es temporal
# Cuando compruebe que funciona se eliminará esta celda.
y_train = y_train[:1000000]
y_train.shape

torch.Size([1000000])

Carga de datos de prueba

In [6]:
X_test = torch.load('./data/X_test.pt')
y_test = torch.load('./data/y_test.pt')

In [7]:
# La reducción del dataset de prueba es temporal.
# Cuando compruebe que funciona se eliminará esta celda.
X_test = X_test[:500000]
X_test.shape

torch.Size([500000, 16])

In [8]:
# La reducción del dataset de prueba es temporal.
# Cuando compruebe que funciona se eliminará esta celda.
y_test = y_test[:500000]
y_test.shape

torch.Size([500000])

## Embedding de títulos

In [9]:
# https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding
embeddings_matrix = torch.load('./data/embeddings_matrix.pt')
embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                          padding_idx=0)

## Construcción del Dataset

In [10]:
train_dataset = MeLiChallengeDataset(X_train, y_train)
test_dataset = MeLiChallengeDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          drop_last=False)
i = 0
for data in tqdm(train_loader):
    i += 1
print(f'Recorrida exitosa de {i} batches de entrenamiento.')

test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         drop_last=False)
i = 0
for data in tqdm(test_loader):
    i += 1
print(f'Recorrida exitosa de {i} batches de prueba.')

  0%|          | 0/10000 [00:00<?, ?it/s]

Recorrida exitosa de 10000 batches de entrenamiento.


  0%|          | 0/5000 [00:00<?, ?it/s]

Recorrida exitosa de 5000 batches de prueba.


## Construcción del Modelo

In [11]:
modelo = MeLiChallengeClassifier(embeddings)

## Algoritmo de Optimización

In [12]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.001, momentum=0.9)

## Entrenamiento del modelo

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Utilizando {device}')
modelo.to(device)

Utilizando cuda


MeLiChallengeClassifier(
  (embeddings): Embedding(50002, 300, padding_idx=0)
  (hidden1): Linear(in_features=300, out_features=300, bias=True)
  (hidden2): Linear(in_features=300, out_features=500, bias=True)
  (output): Linear(in_features=500, out_features=632, bias=True)
)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader):
        X, y = data['data'].to(device), data['target'].to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## Evaluación del Modelo

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data in dataloader:
            X, y = data['data'].to(device), data['target'].to(device)
            pred = model(X)
            # accuracy = balanced_accuracy_score(y, pred)
    # print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%")

In [18]:
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, modelo, loss_function, optimizer)
    test(test_loader, modelo, loss_function)
print("Done!")

Epoch 1
-------------------------------
loss: 6.449830  [    0/1000000]
loss: 6.449461  [100000/1000000]
loss: 6.448513  [200000/1000000]
loss: 6.447981  [300000/1000000]
loss: 6.448061  [400000/1000000]
loss: 6.447908  [500000/1000000]
loss: 6.446182  [600000/1000000]
loss: 6.444788  [700000/1000000]
loss: 6.446178  [800000/1000000]
loss: 6.443645  [900000/1000000]
Done!


## ¿Por qué sospecho que no funciona?

La red neuronal tiene como salida un vector de 632 componentes el cuel es comparado con un solo componente, la etiqueta asocada a la clasificación del título. Entiendo que allí hay una incompatibilidad que el código acepta (no da error) pero que evidentemente no permite que la red aprenda.